# Setup

In [ ]:
videos_dir = None
save_dir = None

In [ ]:
import os

dir_path = os.getcwd()

if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
        
if not save_dir:
    save_dir = f'{dir_path}/scene_JSON'

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

fatal: destination path 'TransNetV2' already exists and is not an empty directory.
Updated Git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/feat/integrate-information
Checking out LFS objects: 100% (3/3), 0 B | 0 B/s, done.                        


In [ ]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from TransNetV2.inference.transnetv2 import TransNetV2

2024-09-01 17:11:03.567206: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 17:11:03.677411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 17:11:03.718195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 17:11:03.730786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 17:11:03.815616: I tensorflow/core/platform/cpu_feature_guar

# Parse video info

In [ ]:
def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]  # L01, L02 for example
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = f'{videos_dir}/Videos_{data_part}/video'
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = f'{data_part_path}/{video_path}'
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths

# Inference

In [ ]:
def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def predict_scenes_and_save(model, all_video_paths, save_dir):
    """Predict scenes for each video and save the results as JSON files."""
    create_directory(save_dir)

    for key, video_paths_dict in tqdm(all_video_paths.items()):      
        video_ids = sorted(video_paths_dict.keys())
        key_dir = os.path.join(save_dir, key)
        create_directory(key_dir)

        for video_id in tqdm(video_ids, desc=f"Processing {key}"):            
            video_path = video_paths_dict[video_id]
            _, single_frame_predictions, _ = model.predict_video(video_path)

            # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
            scenes = model.predictions_to_scenes(single_frame_predictions)

            with open(os.path.join(key_dir, f"{video_id}.json"), 'w') as f:
                json.dump(scenes.tolist(), f)

In [ ]:
model = TransNetV2()
all_video_paths = parse_video_info(videos_dir)
predict_scenes_and_save(model, all_video_paths, save_dir)

[TransNetV2] Using weights from /home/heigatvu/my-project/competition/HCMC-AI/pipeline-hcm-ai/notebooks/data_extraction/transnet/TransNetV2/inference/transnetv2-weights/.


I0000 00:00:1725185465.780996   14906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725185465.783984   14906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725185465.784107   14906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725185465.785142   14906 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

[TransNetV2] Extracting frames from /home/heigatvu/my-project/competition/HCMC-AI/pipeline-hcm-ai/notebooks/data_extraction/dataset/AIC_Video/Videos_L01/video/L01_V001.mp4


2024-09-01 17:11:23.413538: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1725185483.540632   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.575838   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.577931   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.589192   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.590010   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.590970   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.664909   15863 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725185483.668290   15863 gpu_t

[TransNetV2] Processing video frames 31550/31665

[TransNetV2] Processing video frames 31665/31665
[TransNetV2] Extracting frames from /home/heigatvu/my-project/competition/HCMC-AI/pipeline-hcm-ai/notebooks/data_extraction/dataset/AIC_Video/Videos_L01/video/L01_V002.mp4
[TransNetV2] Processing video frames 24337/24337


100%|██████████| 1/1 [00:58<00:00, 58.17s/it]
